# Home Automation Project

## Lights

The intensity of lights is dependent on the daylight.
The lights would get turned OFF gradually before sunrise, and they would get turned ON gradually before sunset.

In [19]:
from astral import LocationInfo
import pytz
from astral.geocoder import database, lookup

# Set up location information
tz_IND = pytz.timezone('Asia/Kolkata')
tz_LA = pytz.timezone('America/Los_Angeles')
# city = LocationInfo("Pune", "India", "Asia/Kolkata", 18.493255, 73.827113)
city = LocationInfo("Costa Mesa", "USA", "America/Los_Angeles", 33.692572, -117.878945)
# time_zone = lookup("Mumbai", database()).timezone
time_zone = lookup("Los_Angeles", database()).timezone
# tz = tz_IND
tz = tz_LA

# Define time_zone variable
time_zone = None

# Display location information
print((
    f"Information for {city.name}/{city.region}\n"
    f"Timezone: {city.timezone}\n"
    f"Latitude: {city.latitude:.02f}; Longitude: {city.longitude:.02f}\n"
))

Information for Costa Mesa/USA
Timezone: America/Los_Angeles
Latitude: 33.69; Longitude: -117.88



## Philips Hue

In [20]:
# Get sunrise and sunset information
from astral.sun import sun
s = sun(city.observer, tzinfo=time_zone)
print((
    f'Sunrise: {s["sunrise"].time()}\n'
    f'Sunset:  {s["sunset"].time()}\n'
))
sunrise = s['sunrise'].time()
sunset = s['sunset'].time()

# Get current time
from datetime import datetime
now = datetime.now(tz)
# print(f'Current time: {now.time()}')

# Get time remaining for sunset and sunrise, and set intensity of light accordingly
from datetime import datetime, date, timedelta
one_hour = timedelta(hours=1)
one_minute = timedelta(minutes=1)
zero_seconds = timedelta(seconds=0)

# Define brightness variable
brightness = 254

# Time to sunset
time_to_sunset = datetime.combine(date.today(), sunset) - datetime.combine(date.today(), now.time())
if time_to_sunset < one_hour and time_to_sunset > zero_seconds: # Less than 1 hr remaining for sunset
    # Calculate brightness of light in range 0-254
    brightness = ((one_hour - time_to_sunset)/one_minute)*254/60
    print(f'Brightness: {brightness}')
else:
    # Print time to sunset
    if time_to_sunset < zero_seconds:
        time_to_sunset = datetime.combine((date.today()+timedelta(days=1)), sunset) - datetime.combine(date.today(), now.time())
    print(f'Time to sunset: {time_to_sunset}')

# Time to sunrise
time_to_sunrise = datetime.combine(date.today(), sunrise) - datetime.combine(date.today(), now.time())
if time_to_sunrise < one_hour and time_to_sunrise > zero_seconds: # Less than 1 hr remaining for sunrise
    # Calculate brightness of light in range 0-254
    brightness = (time_to_sunrise/one_minute)*254/60
    print(f'Brightness: {brightness}')
else:
    # Print time to sunrise
    if time_to_sunrise < zero_seconds:
        time_to_sunrise = datetime.combine((date.today()+timedelta(days=1)), sunrise) - datetime.combine(date.today(), now.time())
    print(f'Time to sunrise: {time_to_sunrise}\n')

import json
import requests
import urllib3
urllib3.disable_warnings()
hue_bridge_ip = '192.168.50.234'
user = 'kYezkr7YNGozEVSHkfneoHx3gO44ff8Ns0Qn9eKi'
url = 'https://'+hue_bridge_ip+'/api/'+user+'/lights'

# Get light status
response = requests.get(url, verify=False)
# print("Response code = ",response.status_code)
data = json.loads(response.text)    # Convert json data to python dictionary
# print(json.dumps(data, indent=4))   # Pretty print json data
light_id = list(data.keys())        # Get the light ids
# print("Light IDs = ", light_id)

# Get the hue of the input color name
from colour import Color
user_color = input("Enter color name: ")
color = Color(user_color)
# Get hue, saturation
sat = color.saturation
hue = color.hue
hue = int(hue*65535) # Hue is in range 0-1, convert it to 0-65535
sat = int(sat*254)  # Saturation is in range 0-1, convert it to 0-254

# Set the light state to ON
# loop through all lights
for light in light_id:
    url = 'https://'+hue_bridge_ip+'/api/'+user+'/lights/'+light+'/state'
    # print(url)
    headers = {
        'content-type': "application/json",
        'cache-control': "no-cache"
    }
    data ={'on': True, 'sat': sat, 'bri': brightness, 'hue': hue}
    response = requests.put(url, headers=headers, data=json.dumps(data), verify=False)
    # print(response.status_code)
    # Display json data
    # data = json.loads(response.text)
    # print(json.dumps(data, indent=4))   # Pretty print json data

Sunrise: 06:02:10.884531
Sunset:  19:53:21.900155

Time to sunset: 23:39:47.977455
Time to sunrise: 9:48:36.961831

Light IDs =  ['12', '13', '14']


### Motion detected lights

There would be a LED light strip for each room which would light up subtlely and will be triggered by motion.

1. Routines:
    * If time is more than sunset and less than sunrise, then trigger lights that leads to the bathroom. These lights would be on the floor level.
    * A device (proxi sensor) to keep track of number of people in the house and in the room.
    

## Camera

This would keep track of number of people in the house.


## House AI

It would be updated with the number of people in the house, and in which room they are in. 

There would be microphones/with camera in each room listening to the commands And sending them to the central server.

The facial recognition will update number of persons in each room. (no facial recognition (for now)).

The voice would not have a recognition system (for now)

